In [49]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List


# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_SOURCE_FILE = os.path.join(DATA_DIR, "airport_source.csv")
# change to match your gcloud project 
PROJECT_NAME = "deb-01-372116"
DATASET_NAME = "airline_ticket_processor"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'f_tickets': {
        'table_name': 'f_tickets',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED'),
            bigquery.SchemaField('airline_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('confirmation', 'string', mode='NULLABLE'),
            bigquery.SchemaField('ticket_date', 'string', mode='NULLABLE'),
            bigquery.SchemaField('org_iata_dest_iata', 'string', mode='NULLABLE'),
            bigquery.SchemaField('price', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('seat', 'string', mode='NULLABLE'),
            bigquery.SchemaField('status', 'string', mode='NULLABLE'),
            bigquery.SchemaField('UUID_SK', 'STRING', mode='NULLABLE'),
        ],
    },
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

DEBUG:root:Creating bigquery client
INFO:root:Setup Completed


In [50]:
#load airlines data
# receipts data file name
filename = DEFAULT_SOURCE_FILE
logger.debug(f"attempting to process: {filename}")

# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
df = pd.read_csv(filename, header=0)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# assign & remember receipts dataframe
source_df = df

DEBUG:root:attempting to process: ./data/airport_source.csv
INFO:root:loaded 4096 rows from: ./data/airport_source.csv


In [51]:
#  SCD2_d_passengers table query
query = f"""
SELECT
  UUID_SK,
  email
FROM 
  `{PROJECT_NAME}.{DATASET_NAME}.SCD2_d_passengers`
"""
# query bigquery table into dataframe
pdf = client.query(query).to_dataframe()
# index by product_name for the join
pdf = pdf.set_index(keys='email')
logger.info(f"query {len(pdf.index)} rows from SCD2_d_passengers table")


# drop product_id and customer_id columns if they already exist
#   this line will allow us to re-run this cell
df = df.drop(columns=['UUID_SK'], errors='ignore')

logger.debug(f"preparing for join")
# join with the main dataframe to get product_id and customer_id
df = df.join(pdf, on='email', how='inner')
logger.info(f"looked up UUID_SK")

# sort back by receipt number
logger.debug(f"sorting back by receipt id to get the rows")
df = df.sort_values(by=['UUID_SK'])


logger.debug(f"dataframe:")

INFO:root:query 128 rows from SCD2_d_passengers table
DEBUG:root:preparing for join
INFO:root:looked up UUID_SK
DEBUG:root:sorting back by receipt id to get the rows
DEBUG:root:dataframe:


In [52]:
#define the load_table function
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [53]:
##----------------load d_airlines---------------------
# get unique receipts (not line items)
f_tickets = df[['eticket_num','airline_iata', 'confirmation', 'ticket_date','org_iata_dest_iata', 'price', 'seat','status','UUID_SK']].drop_duplicates(keep='first')
logger.info(f"Preparing {len(f_tickets.index)} unique receipts to load to bigquery")
f_tickets = f_tickets.astype({"UUID_SK": str})

display(f_tickets.head())

# load to bigquery
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['f_tickets']['table_name']}"
schema = TABLE_METADATA['f_tickets']['schema']
load_table(f_tickets, client, table_name, schema)

INFO:root:Preparing 4096 unique receipts to load to bigquery


,eticket_num,airline_iata,confirmation,ticket_date,org_iata_dest_iata,price,seat,status,UUID_SK
2803,365-706800-1366,AF,MB30LG,2022-03-21,SIN-CTA,747.18,16B,active,06484a3e-d210-46df-9c6c-7b8407382f82
3074,767-109860-3831,DD,5K6A7L,2022-03-21,MXP-BEL,986.73,14B,active,06484a3e-d210-46df-9c6c-7b8407382f82
3071,047-696377-8192,MW,0X6GOS,2022-03-24,SLC-COR,548.67,16G,active,06484a3e-d210-46df-9c6c-7b8407382f82
3057,628-192959-1240,HR,L5XNC7,2022-03-24,BAQ-HKG,487.46,18F,active,06484a3e-d210-46df-9c6c-7b8407382f82
3002,491-437150-3688,FD,DUMEIG,2022-03-21,DAC-STM,339.64,32E,active,06484a3e-d210-46df-9c6c-7b8407382f82


INFO:root:loading table: 'deb-01-372116.airline_ticket_processor.f_tickets'
INFO:root:loaded 4096 rows into deb-01-372116:airline_ticket_processor.f_tickets
